In [1]:
import avocado.validation_tests
import avocado.utils as utils
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
import datetime
import itertools
import numpy as np
import itertools
import tqdm
from sklearn.metrics import confusion_matrix

In [2]:

stations = {
    '001002': "Cabeza Mediana",
    '001003': "Zabala",
    '001004': "Cotos",
    '001005': "Alameda",
    '001006': "Las Hoyas",
    '001007': "Dos Hermanas",
    '001101': "El Paular",
    '001102': "Boca del Asno",
    '001103': "Pedriza",
    '001104': "Las Dehesas",
    '001201': "Pico Penalara" 
}
"""
stations = {
    '001002': "Cabeza Mediana",
}
"""

sampling_frequency = "10min"

variables = [
    'TMPA01#10MN.AVG',
    'RHMA01#10MN.AVG',
    'WSPD01#10MN.AVG',
    'WDIR01#10MN.AVG',
    'PCNR01#10MN.SUM',
    'RADS01#10MN.AVG'
]

years = [str(y) for y in range(2000, 2024)]

In [3]:
dir_path = "/mnt/VMS/TIMESERIES/VERDE/"

In [4]:
is_station = lambda x: True if x in list(stations.keys()) else False
is_variable = lambda x: True if x in variables else False

def get_data_filepaths(dir_path, stations):
    
    station_files = stations.copy()
    for k, v in station_files.items():
        station_files[k] = []

    for root, directories, files in os.walk(dir_path):
        if root.split("/")[-1] in years:
            for file in files:
             
                splitted_filename = file.split("_")
                
                code = splitted_filename[0]
                variable = splitted_filename[1]
                initial_date = splitted_filename[2]
                final_date = splitted_filename[3]
                generation_date = splitted_filename[4]
                data_type = splitted_filename[5].replace(".csv", "")

                if is_station(code) and is_variable(variable):
                    station_files[code].append("/".join([root, file]))
                    
    return station_files


def get_stations_data(path, stations, variables):
    
    station_files = get_data_filepaths(path, stations)

    all_stations_data = {}
    validation_columns = ["ValTag_" + v for v in variables]
    station_data_cols = variables.extend(validation_columns)

    index = pd.date_range(start=datetime.datetime(2000, 1, 1, 0, 0), end=datetime.datetime(2023, 12, 31, 23, 50), freq="10min")

    for code, filepaths in station_files.items():
        station_data = pd.DataFrame(columns=station_data_cols, index=index)
        for filepath in tqdm.tqdm(filepaths, desc="Loading " + code):
            variable = filepath.split("/")[-1].split("_")[1]
        
            df = pd.read_csv(filepath, index_col=0, skiprows=5)
            df = df.rename(columns={"ValTag": "ValTag_" + variable})
            df.index = pd.to_datetime(df.index)
            df.index = df.index.round(freq="10min")

            station_data.loc[df.index, [variable, "ValTag_" + variable]] = df

        all_stations_data[code] = station_data

    return all_stations_data


def delete_invalid_data(df, variables, timestamps):
    for variable in variables:
        if variable in df.columns:
            df_to_clean = df.loc[timestamps, ["ValTag_" + variable]]
            timestamps_to_clean = df_to_clean[df_to_clean["ValTag_" + variable] == "V"].index
            df.loc[timestamps_to_clean, variable] = np.nan
            df = df.drop(["ValTag_" + variable], axis=1)
    return df

def combinations_of(list_1, list_2):
    combinations=[]
    permutations = itertools.permutations(list_1, len(list_2))
    for combination in permutations:
        zipped = zip(combination, list_2)
        combinations.extend(list(zipped))
    combinations = list(set(combinations))
    return combinations

def get_automatic_validation_tags(all_stations_data):

    tests = ["IV", "CC", "VC", "IC"]
    variable_names = ["TMPA", "RHMA", "WDIR", "WSPD", "RADS01", "PCNR"]
    
    tags = combinations_of(variable_names, tests)
    tags = ["_".join(c) for c in tags]

    stations_validation = {}

    for station, station_data in all_stations_data.items():

        validation_tags = pd.DataFrame(index=station_data.index, columns=tags)
        print(validation_tags)
        for year in tqdm.tqdm(years, desc="Validating " + station):
            
            validation_start = [int(year), 1, 1, 0, 0]
            validation_end = [int(year), 12, 31, 23, 59]

            training_period = [i for i in station_data.index if i.year != year]
            data_clean_training = delete_invalid_data(station_data, variables=variables, timestamps=training_period) 

            variables_to_validate = []
            for col in data_clean_training.columns:
                variable_name = col.split("#")[0]
                if variable_name not in ["RADS01", "RADS02", "RADL01", "RADL02"]:
                    variable_name = variable_name.split("0")[0]
                variables_to_validate.append(variable_name)    

            data_clean_training.columns = variables_to_validate

            if "IV" in tests:
                data_clean_training = data_clean_training.validate.impossible_values(
                    variables_to_validate,
                    start=validation_start,
                    end=validation_end,
                    freq=sampling_frequency
                )

            if "CC" in tests:
                data_clean_training = data_clean_training.validate.climatological_coherence(
                    variables_to_validate,
                    percentiles=[0.01, 0.99],
                    start=validation_start,
                    end=validation_end,
                    freq=sampling_frequency
                )

            if "VC" in tests:
                data_clean_training = data_clean_training.validate.variance_test(
                    variables_to_validate,
                    '1D',
                    percentiles=[0.01, 0.99],
                    start=validation_start,
                    end=validation_end,
                    freq=sampling_frequency
                )

            if "SC" in tests:
                data_clean_training = data_clean_training.validate.spatial_coherence(
                    reference_observations,
                    variables_to_validate,
                    min_corr=0.8,
                    percentiles=[0.01, 0.99],
                    start=validation_start,
                    end=validation_end,
                    freq=sampling_frequency
                )

            if "IC" in tests:
                cleared_radiation_observations = utils.Preprocess(data_clean_training).clear_low_radiance()
                try:
                    data_clean_training = cleared_radiation_observations.validate.internal_coherence(
                        percentiles=[0.01, 0.99],
                        start=validation_start,
                        end=validation_end,
                        freq=sampling_frequency,
                        plot_eofs=False,
                        plot_reconstruction=False
                    )
                except:
                    data_clean_training[[v + "_IC" for v in variable_names]] = np.nan

        
            validation_tags_year = data_clean_training.drop(variables_to_validate, axis=1)
            print(validation_tags_year)
            print(validation_tags)
            validation_tags.loc[validation_tags_year.index] = validation_tags_year
            
        stations_validation[station] = validation_tags

    return stations_validation

def get_manual_validation_tags(all_stations_data):
    manual_tags = {}
    for station, station_data in all_stations_data.items():
        validation_columns = [col for col in station_data.columns if "ValTag" in col]
        station_manual_tags = station_data[validation_columns].applymap(lambda x: 1 if x == "V" else 0)
        manual_tags[station] = station_manual_tags
    return manual_tags

# CROSS-VALIDATION
Each year is validated leaving it out from the training dataset. For the training dataset, we use the already validated data.

In [5]:
all_stations_data = get_stations_data(path=dir_path, stations=stations, variables=variables)

Loading 001201: 100%|███████████████████████████| 57/57 [00:04<00:00, 12.76it/s]


In [6]:
manual_validation_tags = get_manual_validation_tags(all_stations_data)

In [7]:
manual_validation_tags

{'001002':                      ValTag_RADS01#10MN.AVG  ValTag_WSPD01#10MN.AVG  \
 2000-01-01 00:00:00                       0                       1   
 2000-01-01 00:10:00                       0                       1   
 2000-01-01 00:20:00                       0                       1   
 2000-01-01 00:30:00                       0                       1   
 2000-01-01 00:40:00                       0                       1   
 ...                                     ...                     ...   
 2023-12-31 23:10:00                       0                       0   
 2023-12-31 23:20:00                       0                       0   
 2023-12-31 23:30:00                       0                       0   
 2023-12-31 23:40:00                       0                       0   
 2023-12-31 23:50:00                       0                       0   
 
                      ValTag_RHMA01#10MN.AVG  ValTag_WDIR01#10MN.AVG  \
 2000-01-01 00:00:00                       0        

In [8]:
automatic_validation_tags = get_automatic_validation_tags(all_stations_data)

                    TMPA_IV PCNR_CC RHMA_IC RADS01_VC WDIR_CC WSPD_IV TMPA_VC  \
2000-01-01 00:00:00     NaN     NaN     NaN       NaN     NaN     NaN     NaN   
2000-01-01 00:10:00     NaN     NaN     NaN       NaN     NaN     NaN     NaN   
2000-01-01 00:20:00     NaN     NaN     NaN       NaN     NaN     NaN     NaN   
2000-01-01 00:30:00     NaN     NaN     NaN       NaN     NaN     NaN     NaN   
2000-01-01 00:40:00     NaN     NaN     NaN       NaN     NaN     NaN     NaN   
...                     ...     ...     ...       ...     ...     ...     ...   
2023-12-31 23:10:00     NaN     NaN     NaN       NaN     NaN     NaN     NaN   
2023-12-31 23:20:00     NaN     NaN     NaN       NaN     NaN     NaN     NaN   
2023-12-31 23:30:00     NaN     NaN     NaN       NaN     NaN     NaN     NaN   
2023-12-31 23:40:00     NaN     NaN     NaN       NaN     NaN     NaN     NaN   
2023-12-31 23:50:00     NaN     NaN     NaN       NaN     NaN     NaN     NaN   

                    PCNR_IC

Validating 001002:   0%|                                 | 0/24 [00:00<?, ?it/s]/usr/lib/python3/dist-packages/dask/array/routines.py:1526: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
/usr/lib/python3/dist-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


                     RADS01_IV  WSPD_IV  RHMA_IV  WDIR_IV  TMPA_IV  PCNR_IV  \
2000-01-01 00:00:00        0.0      0.0      0.0      0.0      0.0      0.0   
2000-01-01 00:10:00        0.0      0.0      0.0      0.0      0.0      0.0   
2000-01-01 00:20:00        0.0      0.0      0.0      0.0      0.0      0.0   
2000-01-01 00:30:00        0.0      0.0      0.0      0.0      0.0      0.0   
2000-01-01 00:40:00        0.0      0.0      0.0      0.0      0.0      0.0   
...                        ...      ...      ...      ...      ...      ...   
2023-12-31 23:10:00        NaN      NaN      NaN      NaN      NaN      NaN   
2023-12-31 23:20:00        NaN      NaN      NaN      NaN      NaN      NaN   
2023-12-31 23:30:00        NaN      NaN      NaN      NaN      NaN      NaN   
2023-12-31 23:40:00        NaN      NaN      NaN      NaN      NaN      NaN   
2023-12-31 23:50:00        NaN      NaN      NaN      NaN      NaN      NaN   

                     RHMA_CC  WSPD_CC  WDIR_CC  PCN

Validating 001002:   4%|█                        | 1/24 [01:07<25:47, 67.28s/it]/usr/lib/python3/dist-packages/dask/array/routines.py:1526: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
/usr/lib/python3/dist-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


                     RADS01_IV  WSPD_IV  RHMA_IV  WDIR_IV  TMPA_IV  PCNR_IV  \
2000-01-01 00:00:00        NaN      NaN      NaN      NaN      NaN      NaN   
2000-01-01 00:10:00        NaN      NaN      NaN      NaN      NaN      NaN   
2000-01-01 00:20:00        NaN      NaN      NaN      NaN      NaN      NaN   
2000-01-01 00:30:00        NaN      NaN      NaN      NaN      NaN      NaN   
2000-01-01 00:40:00        NaN      NaN      NaN      NaN      NaN      NaN   
...                        ...      ...      ...      ...      ...      ...   
2023-12-31 23:10:00        NaN      NaN      NaN      NaN      NaN      NaN   
2023-12-31 23:20:00        NaN      NaN      NaN      NaN      NaN      NaN   
2023-12-31 23:30:00        NaN      NaN      NaN      NaN      NaN      NaN   
2023-12-31 23:40:00        NaN      NaN      NaN      NaN      NaN      NaN   
2023-12-31 23:50:00        NaN      NaN      NaN      NaN      NaN      NaN   

                     RHMA_CC  WSPD_CC  WDIR_CC  PCN

...

                     RADS01_IV  PCNR_IV  RHMA_IV  WDIR_IV  TMPA_IV  WSPD_IV  \
2000-01-01 00:00:00        NaN      NaN      NaN      NaN      NaN      NaN   
2000-01-01 00:10:00        NaN      NaN      NaN      NaN      NaN      NaN   
2000-01-01 00:20:00        NaN      NaN      NaN      NaN      NaN      NaN   
2000-01-01 00:30:00        NaN      NaN      NaN      NaN      NaN      NaN   
2000-01-01 00:40:00        NaN      NaN      NaN      NaN      NaN      NaN   
...                        ...      ...      ...      ...      ...      ...   
2023-12-31 23:10:00        0.0      0.0      0.0      0.0      0.0      0.0   
2023-12-31 23:20:00        0.0      0.0      0.0      0.0      0.0      0.0   
2023-12-31 23:30:00        0.0      0.0      0.0      0.0      0.0      0.0   
2023-12-31 23:40:00        0.0      0.0      0.0      0.0      0.0      0.0   
2023-12-31 23:50:00        0.0      0.0      0.0      0.0      0.0      0.0   

                     WSPD_CC  RHMA_CC  WDIR_CC  PCN

Validating 001004: 100%|████████████████████████| 24/24 [27:14<00:00, 68.09s/it]


                    TMPA_IV PCNR_CC RHMA_IC RADS01_VC WDIR_CC WSPD_IV TMPA_VC  \
2000-01-01 00:00:00     NaN     NaN     NaN       NaN     NaN     NaN     NaN   
2000-01-01 00:10:00     NaN     NaN     NaN       NaN     NaN     NaN     NaN   
2000-01-01 00:20:00     NaN     NaN     NaN       NaN     NaN     NaN     NaN   
2000-01-01 00:30:00     NaN     NaN     NaN       NaN     NaN     NaN     NaN   
2000-01-01 00:40:00     NaN     NaN     NaN       NaN     NaN     NaN     NaN   
...                     ...     ...     ...       ...     ...     ...     ...   
2023-12-31 23:10:00     NaN     NaN     NaN       NaN     NaN     NaN     NaN   
2023-12-31 23:20:00     NaN     NaN     NaN       NaN     NaN     NaN     NaN   
2023-12-31 23:30:00     NaN     NaN     NaN       NaN     NaN     NaN     NaN   
2023-12-31 23:40:00     NaN     NaN     NaN       NaN     NaN     NaN     NaN   
2023-12-31 23:50:00     NaN     NaN     NaN       NaN     NaN     NaN     NaN   

                    PCNR_IC

Validating 001005:   0%|                                 | 0/24 [00:30<?, ?it/s]


AttributeError: Must have RADS01

# YEARLY TIME SERIES

In [ ]:
for station, manual_tags in manual_validation_tags.items():
    
    n_elements = manual_tags.groupby(manual_tags.index.year).agg('count')
    yearly_tags = manual_tags.groupby(manual_tags.index.year).sum()

    yearly_tags = yearly_tags.applymap(lambda x: np.nan if x == 0 else x)
    
    n_elements = n_elements[~yearly_tags.isna()] 
    
    total_elements = n_elements.sum(axis=1)
    total_tags = yearly_tags.sum(axis=1)
    
    tag_percentage = yearly_tags/n_elements
    total_tag_percentage = total_tags/total_elements

    fig = plt.figure(figsize=(20, 4))
    ax = fig.subplots()
    tag_percentage.plot(ax=ax, marker='o', alpha=0.6)
    total_tag_percentage.plot(ax=ax, marker='o', color='black', label="Total")
    ax.legend(bbox_to_anchor=(1, 1))
    ax.set_ylabel(station + " percentage of valid data [%]")
    ax.set_title(station)
    ax.grid()

In [ ]:
automatic_validation_tags

In [ ]:
tests = ["CC", "VC", "IC"]
combinations = []
for i in range(1, len(tests) + 1):
    combinations.extend(list(itertools.combinations(tests, i)))

combinations_as_str = ["".join(c) for c in combinations]

# Seasonal Tags

In [ ]:
for station, manual_tags in manual_validation_tags.items():

    yearly_tags = manual_tags.groupby(manual_tags.index.year).sum()
    yearly_tags = yearly_tags.applymap(lambda x: np.nan if x == 0 else 1)
    
    monthly_tags = manual_tags.resample("1m").sum()
    n_elements = manual_tags.resample("1m").count()

    monthly_tags = monthly_tags.applymap(lambda x: np.nan if x == 0 else x)
    n_elements = n_elements[~monthly_tags.isna()] 
    
    n_elements = n_elements.groupby(n_elements.index.month).sum()
    monthly_tags = monthly_tags.groupby(monthly_tags.index.month).sum()

    total_elements = n_elements.sum(axis=1)
    total_tags = monthly_tags.sum(axis=1)
    
    tag_percentage = monthly_tags/n_elements
    total_tag_percentage = total_tags/total_elements

    fig = plt.figure(figsize=(5, 5))
    ax = fig.subplots()
    tag_percentage.plot(ax=ax, marker='o', alpha=0.7)
    total_tag_percentage.plot(ax=ax, marker='o', color='black', label="Total")
    ax.legend(bbox_to_anchor=(1, 1))
    ax.set_ylabel(station + " percentage of valid data [%]")
    ax.set_xticks(range(1, 13))
    ax.set_xticklabels(["J", "F", "M", "A", "M", "J", "J", "A", "S", "O", "N", "D"])
    ax.set_title(station)
    ax.grid()

In [ ]:
def plot_combined_tests_skill(df):
    variables = df.index.get_level_values(0)
    for variable in variables:
        
        df_variable = df.loc[variable]
        print(df_variable)
        df_variable = df_variable.reset_index()
        df_variable["n_tests"] = df_variable["test"].apply(lambda x: len(x)/2)
        df_variable = df_variable.sort_values(by=['n_tests'])
        df_variable = df_variable.reset_index().drop(["index", "n_tests"], axis=1)
        df_variable = df_variable.set_index("test")
        print(df_variable)
        df_variable.plot(kind="bar", stacked=True, color=['tab:red', 'tab:green', 'tab:orange', "tab:blue"])
        plt.show()
        

In [ ]:
for station, validation_tags in automatic_validation_tags.items():

    manual_df = manual_validation_tags[station]
    variable_skill = {}
    # validation_combinations_skill = pd.DataFrame(index = combinations_as_str, columns = ["tn", "fp", "fn", "tp"])

    combination_variables = list(set([col.split("_")[0] for col in validation_tags.columns]))

    comb_var = []
    perm = list(itertools.permutations(combination_variables, len(combinations_as_str)))
    for p in perm:
        comb = zip(p, combinations_as_str)
        comb_var.extend(comb)
    comb_var = sorted(list(set(comb_var)))
    index = pd.MultiIndex.from_tuples(comb_var, names=["variable", "test"])

    test_performance = pd.DataFrame(index=index, columns=["True Negative", "False Positive", "False Negative", "True Positive"])

    # Get possible combinations of automatic tests
    for combination in combinations:
        
        combination_name = "".join(combination)
        
        combination_columns = []
        for col in validation_tags.columns:
            tag = col.split("_")[-1]
            if tag in combination:
                combination_columns.append(col)

        combination_df = validation_tags[combination_columns]


        # get the sum of the labels for each variable
        for variable in combination_variables:

            variable_cols_in_automatic = [col for col in combination_df.columns if variable in col]
            variable_cols_in_manual = [col for col in manual_df.columns if variable in col]

            var_combination_df = combination_df[variable_cols_in_automatic]
            var_manual_df = manual_df[variable_cols_in_manual]

            total_var_combination_df = var_combination_df.sum(axis=1).apply(lambda x: x/len(variable_cols_in_automatic))
            # Intersection
            total_var_combination_df = total_var_combination_df.apply(lambda x: 1 if x==len(variable_cols_in_automatic) else 0)
            # Combination
            #total_var_combination_df = total_var_combination_df.apply(lambda x: 1 if x>0 else 0)
            
            common_idx = var_manual_df.index.intersection(total_var_combination_df.index)

            total_var_combination_df = total_var_combination_df.loc[common_idx]
            var_manual_df = var_manual_df.loc[common_idx]

            # Calculate Skill
            tn, fp, fn, tp = confusion_matrix(var_manual_df.values, total_var_combination_df.values).ravel()

            total = tn + fp + fn + tp

            true_negative = tn/total
            false_positive = fp/total
            false_negative = fn/total
            true_positive = tp/total
            # test_performance.loc[variable, combination_name] = [tn, fp, fn, tp] 
            test_performance.loc[variable, combination_name] = [true_negative, false_positive, false_negative, true_positive] 
            
    print(test_performance)
    plot_combined_tests_skill(test_performance)


In [ ]:

    # for variable in df